In [ ]:
{"nbformat": 4, "metadata": {"language_info": {"pygments_lexer": "ipython3", "nbconvert_exporter": "python", "name": "python", "file_extension": ".py", "codemirror_mode": {"name": "ipython", "version": 3}, "mimetype": "text/x-python", "version": "3.6.4"}, "kernelspec": {"display_name": "Python 3", "language": "python", "name": "python3"}}, "cells": [{"cell_type": "code", "outputs": [], "source": ["#!/usr/bin/env python3"], "execution_count": null, "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["# -*- coding: utf-8 -*-"], "execution_count": null, "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["import tensorflow as tf\n", "import numpy as np"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["First we set up the computational graph:"], "metadata": {}}, {"cell_type": "markdown", "source": ["N is batch size; D_in is input dimension;<br>\n", "H is hidden dimension; D_out is output dimension."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["N, D_in, H, D_out = 64, 1000, 100, 10"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Create placeholders for the input and target data; these will be filled<br>\n", "with real data when we execute the graph."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["x = tf.placeholder(tf.float32, shape=(None, D_in))\n", "y = tf.placeholder(tf.float32, shape=(None, D_out))"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Create Variables for the weights and initialize them with random data.<br>\n", "A TensorFlow Variable persists its value across executions of the graph."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["w1 = tf.Variable(tf.random_normal((D_in, H)))\n", "w2 = tf.Variable(tf.random_normal((H, D_out)))"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Forward pass: Compute the predicted y using operations on TensorFlow Tensors.<br>\n", "Note that this code does not actually perform any numeric operations; it<br>\n", "merely sets up the computational graph that we will later execute."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["h = tf.matmul(x, w1)\n", "h_relu = tf.maximum(h, tf.zeros(1))\n", "y_pred = tf.matmul(h_relu, w2)"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Compute loss using operations on TensorFlow Tensors"], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["loss = tf.reduce_sum((y - y_pred) ** 2.0)"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Compute gradient of the loss with respect to w1 and w2."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Update the weights using gradient descent. To actually update the weights<br>\n", "we need to evaluate new_w1 and new_w2 when executing the graph. Note that<br>\n", "in TensorFlow the the act of updating the value of the weights is part of<br>\n", "the computational graph; in PyTorch this happens outside the computational<br>\n", "graph."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["learning_rate = 1e-6\n", "new_w1 = w1.assign(w1 - learning_rate * grad_w1)\n", "new_w2 = w2.assign(w2 - learning_rate * grad_w2)"], "execution_count": null, "metadata": {}}, {"cell_type": "markdown", "source": ["Now we have built our computational graph, so we enter a TensorFlow session to<br>\n", "actually execute the graph."], "metadata": {}}, {"cell_type": "code", "outputs": [], "source": ["with tf.Session() as sess:\n", "    # Run the graph once to initialize the Variables w1 and w2.\n", "    sess.run(tf.global_variables_initializer())\n\n", "    # Create numpy arrays holding the actual data for the inputs x and targets\n", "    # y\n", "    x_value = np.random.randn(N, D_in)\n", "    y_value = np.random.randn(N, D_out)\n", "    for _ in range(500):\n", "        # Execute the graph many times. Each time it executes we want to bind\n", "        # x_value to x and y_value to y, specified with the feed_dict argument.\n", "        # Each time we execute the graph we want to compute the values for loss,\n", "        # new_w1, and new_w2; the values of these Tensors are returned as numpy\n", "        # arrays.\n", "        loss_value, _, _ = sess.run([loss, new_w1, new_w2],\n", "                                    feed_dict={x: x_value, y: y_value})\n", "        print(loss_value)"], "execution_count": null, "metadata": {}}], "nbformat_minor": 2}